In [1]:
import pandas as pd
import numpy as np
import multiprocessing as mp
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [35]:
df = pd.read_pickle('311_clean.pkl')
# For inconsistent data
#df = df[(df['difference']>0)&(df.Status!='Open')]
for i in ['Agency','Complaint Type','Location Type','Incident Zip','Borough']:
    t = pd.Categorical(df[i])
    df[i] = t.labels
df['Location Type'] = df['Location Type']+1
df['Incident Zip'] = df['Incident Zip']+1
df['Time'] = pd.qcut(df['difference'], 8, labels=np.arange(1,9,1))

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 591216 entries, 0 to 591215
Data columns (total 18 columns):
index                     591216 non-null int64
Unique Key                591216 non-null int64
Created Date              591216 non-null object
Closed Date               591216 non-null object
Agency                    591216 non-null int8
Agency Name               591216 non-null object
Complaint Type            591216 non-null int16
Descriptor                588073 non-null object
Location Type             591216 non-null int8
Incident Zip              591216 non-null int16
Resolution Description    412357 non-null object
Borough                   591216 non-null int8
Status                    591216 non-null object
Created_TO                591216 non-null datetime64[ns]
Closed_TO                 591216 non-null datetime64[ns]
delta                     591216 non-null timedelta64[ns]
difference                591216 non-null float64
Time                      591216 non-nul

In [37]:
df = df.sample(n=60000)

In [38]:
x_train, x_test, y_train, y_test = train_test_split(np.asarray(df[['Agency','Complaint Type','Location Type',
                'Incident Zip','Borough']]), np.asarray(df.Time), test_size=0.30)

In [39]:
x_train, x_valid, y_train, y_valid  = train_test_split(x_train, y_train, test_size=0.30)

In [40]:
# Random Forest
def RF(x_train,y_train,x_test,y_test,n,summary=False):
    rf = RandomForestClassifier(n_estimators = n)
    rf.fit(x_train, y_train)
    y_pred_test = rf.predict(x_test)
    y_pred_train = rf.predict(x_train)
    if summary==True:
        print y_pred_test.shape
        cm = confusion_matrix(y_test, y_pred_test)
        print cm.shape
        RFcm = cm
        rfMSE = mean_squared_error(y_test, y_pred_test)

        rfACC = (float(cm[0][0] + cm[1][1] + cm[2][2] + cm[3][3] + cm[4][4] + cm[5][5] + cm[6][6] + cm[7][7])) / (len(y_pred_test))

        print "The Accuracy Rate is", rfACC
        print "The Mean Squared Error is", rfMSE
        print "The confusion matrix is:"
        pd.crosstab(y_test, y_pred_test, rownames=['True'], colnames=['Predicted'], margins=True)
    tr = 100.0*accuracy_score(y_train,y_pred_train)
    te = 100.0*accuracy_score(y_test,y_pred_test) 
    return tr,te

In [41]:
performance = {}
for i in range(30):
    i += 1
    a,b = RF(x_train,y_train,x_valid,y_valid,i)
    performance[i] = ((a,b))

In [42]:
ps = pd.DataFrame(performance).T

In [43]:
ps[ps[1]==ps[1].max()]

,0,1
27,62.115646,38.357143


In [44]:
a,b = RF(x_train,y_train,x_test,y_test,27)

In [45]:
print a,b

62.1224489796 38.3833333333
